Import des bibliothèques python :

In [ ]:
import torch 
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import csv 
import pandas as pd
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler


In [ ]:
df = pd.read_csv("dtb.csv")
print(df)
print(df.shape)

class DtSet(Dataset):
    def __init__(self, dataframe):
        self.x = torch.tensor(dataframe.iloc[:, :-1].values, dtype=torch.float32)
        self.y = torch.tensor(dataframe.iloc[:, -1].values, dtype=torch.float32)

    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, ind):
        return self.x[ind], self.y[ind]
    
instance = DtSet(df)

loader = torch.utils.data.DataLoader(instance, batch_size=3, shuffle=True)

print(instance.x, "\n",  instance.y)

In [ ]:
#   MAUVAISE Normalisation des données (Z - score) (creation of noise)

"""for X, Y in loader:
    mean = X.mean()
    std = X.std()
    X = (X - mean) / std"""





In [ ]:
# True normalisation (Z - score)
X = df[["X", "Y", "Z"]].values
Y = df[["res"]].values

scaler_x = StandardScaler()
scaler_y = StandardScaler()
X_tensor = torch.tensor(scaler_x.fit_transform(X), dtype = torch.float32)
Y_tensor = torch.tensor(scaler_y.fit_transform(Y), dtype = torch.float32)


print(X_tensor, "\n", Y_tensor)

In [ ]:
class LinRegModel(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = nn.Linear(in_features, 3),
        self.linear = nn.Linear(3, 6),
        self.linear = nn.Linear(6, 6),
        self.linear = nn.Linear(6, 9),
        self.linear = nn.Linear(9, 12),
        self.linear = nn.Linear(12, 12),
        self.linear = nn.Linear(12, 9),
        self.linear = nn.Linear(9, 9),
        self.linear = nn.Linear(9, 7),
        self.linear = nn.Linear(7, 5),
        self.linear = nn.Linear(5, 3),
        self.linear = nn.Linear(3, 1)


    def forward(self, x):
        return self.linear(x)

In [ ]:
in_features = 3
out_features = 1
model = LinRegModel(in_features, out_features)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.1)


In [ ]:
nb_epochs = 100

for i in range(nb_epochs):
    outputs = model(X_tensor)
    loss = criterion(outputs, Y_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Epoch [{i+ 1}], Loss: {loss.item():.9f}')



In [ ]:

new_x = np.array([[5223, 1, 2025]])
scalar_test = StandardScaler()
new_x_tensor = torch.tensor((new_x), dtype = torch.float32)

model.eval()
with torch.no_grad():
    prediction_norm = model(new_x_tensor)

prediction_denorm = prediction_norm.item() 

print(f"Predicted for X = {new_x}: {(prediction_denorm)}")s